Clean data


In [ ]:
import sys
sys.path.append("../..")

In [ ]:
from ydata_profiling import ProfileReport
from sqlalchemy import text as sql_text
from config.connect_db import ConnectDataBase
import pandas as pd

db = ConnectDataBase()
engine = db.engine()

Query


In [ ]:
def query_table_dtypes(params: str) -> str:
    return f"""       
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = '{params}'
                ORDER BY ordinal_position; 
            """


def query_table(params: str) -> str:
    return f"""
          SELECT * 
          FROM public."{params}";
         """


df_calendar_tdtypes = pd.read_sql(sql=sql_text(
    query_table_dtypes("Calendar_Bronze")), con=engine.connect())
df_listing_tdtypes = pd.read_sql(sql=sql_text(
    query_table_dtypes("Listing_Bronze")), con=engine.connect())
df_reviews_tdtypes = pd.read_sql(sql=sql_text(
    query_table_dtypes("Reviews_Bronze")), con=engine.connect())

Results from bank tables


In [ ]:
df_calendar_silver = pd.read_sql(sql=sql_text(
    query_table("Calendar_Bronze")), con=engine.connect())

df_listing_silver = pd.read_sql(sql=sql_text(
    query_table("Listing_Bronze")), con=engine.connect())

df_reviews_silver = pd.read_sql(sql=sql_text(
    query_table("Reviews_Bronze")), con=engine.connect())

Copy Dataframe origin


In [60]:
df_listing_silver_cln = df_listing_silver.copy()
df_reviews_silver_cln = df_reviews_silver.copy()
df_calendar_silver_cln = df_calendar_silver.copy()

DTypes Dataframe


In [62]:
dtypes = pd.DataFrame(df_listing_silver_cln.dtypes)
df_half_one = dtypes.head(45)
df_half_two = dtypes.iloc[45:]

columns_df = pd.DataFrame(df_listing_silver_cln.select_dtypes(
    include=['object']).dtypes, columns=['name']).reset_index()

columns_df.columns = ['column_name', 'data_type']

mask = df_listing_tdtypes['column_name'].isin(columns_df['column_name'])
filtered_df = df_listing_tdtypes[mask]
mask_text = filtered_df['data_type'] != 'text'
filtered_df = filtered_df.loc[mask_text]['column_name'].tolist()

list_float = ['price']

for column in list_float:
    df_listing_silver_cln[column] = df_listing_silver_cln[column].str.replace(
        '$', '', regex=False).str.replace(',', '', regex=False).astype(float)

for column in filtered_df:
    df_listing_silver_cln[column] = df_listing_silver_cln[column].astype(
        "float")


dtypes = pd.DataFrame(df_listing_silver_cln.dtypes)
df_half_one = dtypes.head(45)
df_half_two = dtypes.iloc[45:]

In [63]:
dtypes = pd.DataFrame(df_reviews_silver_cln.dtypes)

list_datetime = ['date']

for column in list_datetime:
    df_reviews_silver_cln[column] = pd.to_datetime(
        df_reviews_silver_cln[column], format='%Y-%m-%d')

dtypes = pd.DataFrame(df_reviews_silver_cln.dtypes)
display(dtypes)

,0
listing_id,int64
id,int64
date,datetime64[ns]
reviewer_id,int64
reviewer_name,object


In [64]:
dtypes = pd.DataFrame(df_calendar_silver_cln.dtypes)

list_datetime = ['date']
list_float = ['price', 'adjusted_price']

for column in list_datetime:
    df_calendar_silver_cln[column] = pd.to_datetime(
        df_calendar_silver_cln[column], format='%Y-%m-%d')

for column in list_float:
    df_calendar_silver_cln[column] = df_calendar_silver_cln[column].str.replace(
        '$', '', regex=False).str.replace(',', '', regex=False).astype(float)

dtypes = pd.DataFrame(df_calendar_silver_cln.dtypes)
display(dtypes)

,0
listing_id,int64
date,datetime64[ns]
available,object
price,float64
adjusted_price,float64
minimum_nights,float64
maximum_nights,float64


Replace columns


In [65]:
df_listing_silver_cln.columns = df_listing_silver_cln.columns.str.replace(
    r'\W', '_', regex=True)
df_reviews_silver_cln.columns = df_reviews_silver_cln.columns.str.replace(
    r'\W', '_', regex=True)
df_calendar_silver_cln.columns = df_calendar_silver_cln.columns.str.replace(
    r'\W', '_', regex=True)

Handling null values


In [66]:
def check_missing(df):
    res_missing = df.isna().sum()
    res_missing = (res_missing/len(df))*100
    return res_missing[res_missing > 0]

In [67]:
# print("Valores null antes")
# display(pd.DataFrame(check_missing(
#     df_listing_silver_cln).sort_values(ascending=False)))

colums_filnan = pd.DataFrame(check_missing(
    df_listing_silver_cln).sort_values(ascending=False), columns=['name']).reset_index()
colums_filnan.columns = ['column_name', 'percentage']

columns_listing_silver_cln = colums_filnan['column_name'].to_list()

df_listing_silver_cln[columns_listing_silver_cln] = df_listing_silver_cln[columns_listing_silver_cln].fillna(
    "no_info")

In [68]:
colums_filnan = pd.DataFrame(check_missing(
    df_reviews_silver_cln).sort_values(ascending=False), columns=['name']).reset_index()
colums_filnan.columns = ['column_name', 'percentage']

columns_reviews_silver_cln = colums_filnan['column_name'].to_list()

df_reviews_silver_cln[columns_reviews_silver_cln] = df_reviews_silver_cln[columns_reviews_silver_cln].fillna(
    "no_info")

In [69]:
# print("Valores null antes")
# display(pd.DataFrame(check_missing(
#     df_calendar_silver_cln).sort_values(ascending=False)))

colums_filnan = pd.DataFrame(check_missing(
    df_calendar_silver_cln).sort_values(ascending=False), columns=['name']).reset_index()
colums_filnan.columns = ['column_name', 'percentage']

columns_calendar_silver_cln = colums_filnan['column_name'].to_list()

df_calendar_silver_cln[columns_calendar_silver_cln] = df_calendar_silver_cln[columns_calendar_silver_cln].fillna(
    "no_info")

Drop duplicateds


In [70]:
df_listing_silver_cln = df_listing_silver_cln.drop_duplicates()
df_calendar_silver_cln = df_calendar_silver_cln.drop_duplicates()
df_reviews_silver_cln = df_reviews_silver_cln.drop_duplicates()

Removing special characters


In [71]:
pattern = r'<[^>]+>'
df_listing_silver_cln['description'] = df_listing_silver_cln['description'].str.replace(
    pattern, '', regex=True)


df_listing_silver_cln['description'] = df_listing_silver_cln['description'].str.strip(
).str.lower().str.replace('[^a-z\s]', '', regex=True)

Eliminating extremely unbalanced speakers


In [ ]:
# df_listing_silver_cln['maximum_nights'].value_counts(normalize=True).values[0]
# df_listing_silver_cln['maximum_nights'].value_counts(normalize=True)

In [72]:
list_imbalance = []
limit = 0.98
for col in df_listing_silver_cln.columns:
    perc = df_listing_silver_cln[col].value_counts(normalize=True).values[0]
    if perc > limit:
        list_imbalance.append(col)
        # print(col, perc)

In [73]:
df_listing_silver_cln = df_listing_silver_cln.drop(list_imbalance, axis=1)

In [74]:
# Check
list_imbalance = []
limit = 0.98
for col in df_listing_silver_cln.columns:
    perc = df_listing_silver_cln[col].value_counts(normalize=True).values[0]
    if perc > limit:
        list_imbalance.append(col)
        # print(col, perc)

Checking data accuracy


In [75]:
df_listing_silver_cln = df_listing_silver_cln[df_listing_silver_cln['price'] > 0]

In [ ]:
df_listing_silver_cln.describe()

Outliers


In [77]:
# valor do quantil de 2% a esquerda
df_listing_silver_cln[['price', 'maximum_nights']].quantile(.02)

price             100.0
maximum_nights      7.0
Name: 0.02, dtype: float64

In [78]:
# valor do quantil de 2% a direita
df_listing_silver_cln[['price', 'maximum_nights']].quantile(.98)

price             3008.96
maximum_nights    1125.00
Name: 0.98, dtype: float64

In [79]:
print("Quantidade de linhas antes de eliminar os outliers:",
      len(df_listing_silver_cln))

list_quantile = ['price', 'maximum_nights']
df_aux = df_listing_silver_cln.copy()
for col in list_quantile:
    low_limit = df_aux[col].quantile(.02)
    high_limit = df_aux[col].quantile(.98)
    df_listing_silver_cln = df_listing_silver_cln[(df_listing_silver_cln[col] > low_limit) & (
        df_listing_silver_cln[col] < high_limit)]

print("Quantidade de linhas depois de eliminar os outliers:",
      len(df_listing_silver_cln))

Quantidade de linhas antes de eliminar os outliers: 500
Quantidade de linhas depois de eliminar os outliers: 329


In [80]:
df_listing_silver_cln.describe()

,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,price,minimum_nights,maximum_nights,...,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms
count,3.290000e+02,3.290000e+02,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.00000,...,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000
mean,5.166542e+05,2.199125e+06,3.702128,8.054711,-22.967138,-43.207967,3.741641,448.537994,3.851064,148.18541,...,13.927052,28.580547,48.285714,207.705167,77.273556,8.829787,0.577508,3.188450,2.340426,0.848024
std,2.976883e+05,1.661614e+06,3.985036,13.201882,0.026229,0.051410,2.199667,437.521309,6.429662,187.52489,...,11.921267,22.700470,32.791616,127.174338,102.387734,12.627698,1.194849,3.344232,3.264682,1.408166
min,1.787800e+04,6.899700e+04,1.000000,1.000000,-23.031540,-43.490740,1.000000,101.000000,1.000000,8.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.719750e+05,7.922180e+05,1.000000,2.000000,-22.983120,-43.200790,2.000000,195.000000,2.000000,30.00000,...,1.000000,6.000000,17.000000,103.000000,8.000000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,4.802200e+05,1.648634e+06,2.000000,4.000000,-22.974530,-43.191100,3.000000,296.000000,3.000000,90.00000,...,12.000000,26.000000,50.000000,211.000000,37.000000,3.000000,0.000000,2.000000,1.000000,0.000000
75%,8.241020e+05,3.528095e+06,5.000000,9.000000,-22.959750,-43.184960,4.000000,500.000000,4.000000,180.00000,...,28.000000,54.000000,83.000000,328.000000,109.000000,13.000000,1.000000,4.000000,3.000000,1.000000
max,1.016312e+06,9.709135e+06,35.000000,118.000000,-22.840080,-43.161440,16.000000,2600.000000,90.000000,1124.00000,...,30.000000,60.000000,90.000000,365.000000,611.000000,68.000000,8.000000,28.000000,26.000000,6.000000


Saving dataframe in silver table


In [ ]:
df_listing_silver_cln.to_sql(
    'Listing_Silver', engine, if_exists='replace', index=False)

In [ ]:

df_reviews_silver_cln.to_sql(
    'Reviews_Silver', engine, if_exists='replace', index=False)

In [ ]:
df_calendar_silver_cln.to_sql(
    'Calendar_Silver', engine, if_exists='replace', index=False)

Reports profiles


In [82]:
profile_listing = ProfileReport(
    df_listing_silver_cln, title="Pandas Profiling Report")

profile_listing.to_file(f"reports/Listing_Silver.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [84]:
df_reviews_silver_cln = df_reviews_silver_cln.drop('comments', axis=1)


profile_reviews = ProfileReport(
    df_reviews_silver_cln, title="Pandas Profiling Report")

profile_reviews.to_file(f"reports/Reviews_Silver.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [85]:
profile_calendar = ProfileReport(
    df_calendar_silver_cln, title="Pandas Profiling Report")

profile_calendar.to_file(f"reports/Calendar_Silver.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]